In [24]:
import string
import re
import nltk
import pandas as pd
import numpy as np

nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yonti's\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
text = ('G0OgL3e!')

In [16]:
df = pd.read_csv('data/small.csv')
df.head()

,text
0,"hari ini aku masih libur, masih ada waktu semi..."
1,yg ptm hari ini semangaattt
2,dom cilacap udh ptm 100% apa blm?
3,yg dom jabodetabek kalian ptm nya udah 100% kah?
4,"yg ptm yg genap seru bgt, yg ganjil kek nya ba..."


In [30]:
df['lower_text'] = df['text'].str.lower()  # Convert to lowercase
df['remove_number'] = df['lower_text'].str.replace(r'\d+', '') # Remove number
df['punctuation'] = df['remove_number'].str.replace('[^\w\s]', '')  # Remove punctuation
df['tokenized_text'] = df['punctuation'].apply(nltk.word_tokenize) # Tokenize the text
df['freq_token'] = df['tokenized_text'].apply(nltk.FreqDist).apply(lambda x: dict(x)) # Frequency word token
df['stemmed'] = df['punctuation'].apply(lambda x: ' '.join(nltk.corpus.stopwords.words('indonesia') if x.lower() in nltk.corpus.stopwords.words('english') else [x]))

print(df.head)

<bound method NDFrame.head of                                                 text  \
0  hari ini aku masih libur, masih ada waktu semi...   
1                        yg ptm hari ini semangaattt   
2                  dom cilacap udh ptm 100% apa blm?   
3   yg dom jabodetabek kalian ptm nya udah 100% kah?   
4  yg ptm yg genap seru bgt, yg ganjil kek nya ba...   
5             wtp hr ini gue libur tp BESOK PTM 100%   

                                          lower_text  \
0  hari ini aku masih libur, masih ada waktu semi...   
1                        yg ptm hari ini semangaattt   
2                  dom cilacap udh ptm 100% apa blm?   
3   yg dom jabodetabek kalian ptm nya udah 100% kah?   
4  yg ptm yg genap seru bgt, yg ganjil kek nya ba...   
5             wtp hr ini gue libur tp besok ptm 100%   

                                       remove_number  \
0  hari ini aku masih libur, masih ada waktu semi...   
1                        yg ptm hari ini semangaattt   
2               

In [ ]:
lower_text = text.lower()   # Convert text to lower

token_text = re.sub(r"\d+", "", lower_text)    # Remove number
token_text = re.sub('\s+',' ', token_text) # Remove multiple whitespace into single whitespace
token_text = ''.join(c for c in token_text if c not in string.punctuation)      # Remove punctuation
word_tokens = nltk.tokenize.word_tokenize(token_text)    # Tokenize the text
freq_tokens = nltk.FreqDist(word_tokens)    # Frequency word token

stop_words = [word for word in word_tokens if word not in stopwords.words('english')]   # Implement stopwords
    
stemmer = nltk.stem.SnowballStemmer('english')  # Initialized stemmer
stemmed_words = [stemmer.stem(word) for word in stop_words]   # Stem the words

normalized_text = ''.join(stemmed_words)  # Join the words

In [19]:
print(f'Case Folding Result : {lower_text}')
print(f'Remove punctuation, number, multiple whitespace : {token_text}')
print('Tokenizing Result : ',word_tokens)
print('Frequency Token : ',freq_tokens.most_common())
print('Stopword : ',stop_words)
print('Stemmer : ',stemmed_words)
print('Normalized : ',normalized_text)

Case Folding Result : g0ogl3e!
Remove punctuation, number, multiple whitespace : gogle
Tokenizing Result :  ['gogle']
Frequency Token :  [('gogle', 1)]
Stopword :  ['gogle']
Stemmer :  ['gogl']
Normalized :  gogl


In [27]:
df['textblob'] = df['punctuation'].apply(lambda x: TextBlob(x))

# Create a new column in the DataFrame with sentiment analysis scores
df['sentiment'] = df['textblob'].apply(lambda x: x.sentiment.polarity)

# Normalize the sentiment scores to range from -1 (negative) to 1 (positive)
df['sentiment'] = df['sentiment'].apply(lambda x: np.interp(x, [df['sentiment'].min(), df['sentiment'].max()], [-1, 1]))

print(df['sentiment'])

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
Name: sentiment, dtype: float64


In [23]:
analysis = TextBlob(normalized_text)

# Get sentiment polarity (-1 to 1: negative to positive)
sentiment = analysis.sentiment.polarity
if sentiment > 0: 
    print("Positive sentiment")
elif sentiment < 0: 
    print("Negative sentiment")
else: 
    print("Neutral sentiment")

Neutral sentiment
